In [ ]:
# Workflow
# https://techvidvan.com/tutorials/movie-recommendation-system-python-machine-learning/

In [1]:
# imports

import io
import numpy as np
import pandas as pd
import sys
import codecs

import nltk
from gensim.models import Word2Vec, KeyedVectors
from sklearn.manifold import TSNE

%matplotlib inline
import matplotlib.pyplot as plt

import ssl

In [2]:
# override ssl error from trying to reload nltk data

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

In [3]:
# Load in data
watch_history = pd.read_csv('NetflixViewingHistoryDan.csv')
watch_history

,Title,Date
0,Big Fish,3/28/22
1,Monsters vs. Aliens,3/19/22
2,Brand New Cherry Flavor: Limited Series: Tadpo...,3/10/22
3,Brand New Cherry Flavor: Limited Series: I Exist,3/10/22
4,Starship Troopers,3/9/22
...,...,...
973,Battle for Haditha,3/14/15
974,Blackfish,3/14/15
975,The Immigrant,2/24/15
976,Elsa & Fred,2/23/15


In [4]:
# Netflix database
netflix = pd.read_csv('netflix_titles.csv')
netflix

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8803,s8804,TV Show,Zombie Dumb,NaN,NaN,NaN,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g..."
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."


In [30]:
data = []
for item in watch_history['Title']:
    match = netflix[netflix['title']== item]
    if match.shape[0] != 0 and 'Movie' in match['type'].tolist():
        data.append(item)

In [31]:
data

['Total Recall',
 'Someone Great',
 'The Guns of Navarone',
 'Stripes',
 'Klaus',
 'Total Recall',
 "Monty Python's Life of Brian",
 'Pineapple Express',
 'The Mitchells vs. The Machines',
 'America: The Motion Picture',
 'Polar',
 'Rango',
 'The Sum of All Fears',
 'Outside the Wire',
 'Air Force One',
 'Austin Powers: International Man of Mystery',
 'S.W.A.T.',
 'The Death of Stalin',
 '2012',
 'Green Zone',
 'Mad Max',
 'Rush',
 'Quantum of Solace',
 'Casino Royale',
 'Ava',
 'Parker',
 'Operation Finale',
 'The Naked Gun: From the Files of Police Squad!',
 'The Take',
 'Mile 22',
 'Black Sea',
 '6 Underground',
 'The King',
 'The Dawn Wall',
 'A Bridge Too Far',
 'The Old Guard',
 'The Art of War',
 'Angel Has Fallen',
 'Battle: Los Angeles',
 'Act of Valor',
 'Cloudy with a Chance of Meatballs',
 'Extraction',
 'The Ballad of Buster Scruggs',
 'Snowpiercer',
 'Triple Frontier',
 'Code 8',
 'The Do-Over',
 'Speed Is My Need',
 'The Foreigner',
 'Patriot Games',
 'Sherlock Holmes',
